In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer
import librosa
import numpy as np
import soundfile
import json

import random
from python_speech_features import mfcc
import librosa
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from keras import backend as K
from keras.models import Model
from keras.layers import (BatchNormalization, Conv1D, Dense, Input, 
    TimeDistributed, Activation, Bidirectional, SimpleRNN, GRU, LSTM)
from keras.utils.vis_utils import plot_model


import _pickle as pickle
from numpy.lib.stride_tricks import as_strided

from keras.layers import (Input, Lambda)
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint   
import os

In [11]:
# loading scripts
sys.path.insert(1, '../scripts')
sys.path.append("..")
sys.path.append(".")

from data_cleaning import DataCleaner
from data_viz import Data_Viz
import acoustic_modeling as AM

DC = DataCleaner("../logs/preprocessing_notebook.log")
DV = Data_Viz()


In [12]:
# defining parameters

MIN_BATH_SIZE = 20
MFCC_DIME = 13
WINDOW = 20          # in ms
STEP = 10            # in ms
MAX_FREQ = 8000      # in Hz
MODEL_NAME = "RNN_model"

In [13]:
# loading meta data

train_meta = DC.meta_loader("../data/train_meta.csv", "csv")
valid_meta = DC.meta_loader("../data/test_meta.csv", "csv")

print(f"Size of the training set: {len(train_meta)}")
print(f"Size of the validation set: {len(valid_meta)}")

Size of the training set: 800
Size of the validation set: 200


In [14]:
audio_gen = AM.AudioGenerator(train_meta, valid_meta, minibatch_size=MIN_BATH_SIZE,
                       window=WINDOW, step=STEP, max_freq=MAX_FREQ,
                       mfcc_dim=MFCC_DIME)

audio_gen.load_train_data()
audio_gen.load_validation_data()

In [15]:
# The set of characters accepted in the transcription.
supported = """
ሀ ሁ ሂ ሄ ህ ሆ
ለ ሉ ሊ ላ ሌ ል ሎ ሏ
መ ሙ ሚ ማ ሜ ም ሞ ሟ
ረ ሩ ሪ ራ ሬ ር ሮ ሯ
ሰ ሱ ሲ ሳ ሴ ስ ሶ ሷ
ሸ ሹ ሺ ሻ ሼ ሽ ሾ ሿ
ቀ ቁ ቂ ቃ ቄ ቅ ቆ ቋ
በ ቡ ቢ ባ ቤ ብ ቦ ቧ
ቨ ቩ ቪ ቫ ቬ ቭ ቮ ቯ
ተ ቱ ቲ ታ ቴ ት ቶ ቷ
ቸ ቹ ቺ ቻ ቼ ች ቾ ቿ
ኋ
ነ ኑ ኒ ና ኔ ን ኖ ኗ
ኘ ኙ ኚ ኛ ኜ ኝ ኞ ኟ
አ ኡ ኢ ኤ እ ኦ
ኧ
ከ ኩ ኪ ካ ኬ ክ ኮ
ኳ
ወ ዉ ዊ ዋ ዌ ው ዎ
ዘ ዙ ዚ ዛ ዜ ዝ ዞ ዟ
ዠ ዡ ዢ ዣ ዤ ዥ ዦ ዧ
የ ዩ ዪ ያ ዬ ይ ዮ
ደ ዱ ዲ ዳ ዴ ድ ዶ ዷ
ጀ ጁ ጂ ጃ ጄ ጅ ጆ ጇ
ገ ጉ ጊ ጋ ጌ ግ ጐ ጓ ጔ
ጠ ጡ ጢ ጣ ጤ ጥ ጦ ጧ
ጨ ጩ ጪ ጫ ጬ ጭ ጮ ጯ
ጰ ጱ ጲ ጳ ጴ ጵ ጶ ጷ
ፀ ፁ ፂ ፃ ፄ ፅ ፆ ፇ
ፈ ፉ ፊ ፋ ፌ ፍ ፎ ፏ
ፐ ፑ ፒ ፓ ፔ ፕ ፖ
""".split()

char_map = {}
char_map[""] = 0
char_map[" "] = 1
index = 2
for c in supported:
    char_map[c] = index
    index += 1
index_map = {v: k for k, v in char_map.items()}

print(
   # f"The vocabulary is: {char_map} "
    #f"(size ={index_map})"
)

In [16]:
# replace redundant letters

train_meta["Target"] = train_meta["Target"].apply(lambda x: AM.replacer(x))
valid_meta["Target"] = valid_meta["Target"].apply(lambda x: AM.replacer(x))


In [17]:
model = AM.model_1(input_dim=13,
                units=5,
                activation='relu',
                output_dim=len(char_map)+1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 13)]        0         
_________________________________________________________________
rnn (GRU)                    (None, None, 5)           300       
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 5)           20        
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 223)         1338      
_________________________________________________________________
softmax (Activation)         (None, None, 223)         0         
Total params: 1,658
Trainable params: 1,648
Non-trainable params: 10
_________________________________________________________________
None


In [18]:
AM.train(audio_gen, input_to_softmax=model, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

NameError: name 'EPOCHS' is not defined

In [ ]:
AM.predict(audio_gen,14, 'train', model)
_,_,_,raw_pred = predict_raw(audio_gen,14, 'train', model)
raw_pred_char = np.vstack([sorted(char_map.keys(), key=lambda k: char_map[k]) + ['BLANK'], raw_pred])